In [30]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [1]:
#imports
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.streaming import StreamingContext
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
import json
import time
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import explode
from kafka import KafkaConsumer
import copy
import pandas as pd
import csv
import os
from pymongo import MongoClient
from pyspark.ml import PipelineModel
from pyspark.sql.functions import when

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,DateType,StringType

In [2]:
#lancemement de la session spark
spark = (SparkSession
         .builder
         .master('local')
         .appName('twitter')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
         .config("spark.mongodb.input.uri", "mongodb://mongo:27017/db_Booking.*")
         .config("spark.mongodb.output.uri", "mongodb://mongo:27017/db_Booking.*")
         .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0")
         .getOrCreate())

In [3]:
#initialisation du la connexion mongo
client = MongoClient('mongo',27017)

In [4]:
#load de la pipeline saved dans prediction.py
pipe = PipelineModel.load("model_pip")

In [5]:
#df pour recuperer le schema de la df final
df_sc = spark.read.option("delimiter", ";").option("header", True).csv("Twittersc.csv")
# df_sc.show()

In [6]:
#application de la pipeline sur la df
df_sc = pipe.transform(df_sc)
#organisation du schema
df_sc = df_sc.withColumnRenamed("sentiment","date")

In [7]:
#copie du schema de la df de test
s=copy.deepcopy(df_sc.schema)

In [8]:
#Creation de la df final avec le meme schema de fin
df_final = spark.createDataFrame([], s)

In [9]:
#initialistaion du kafkaConsumer
c = KafkaConsumer('test5', bootstrap_servers=['kafka:9093'], api_version=(2,6,0))

In [10]:
#initialisation du schema de recuperation des messages 
cSchema = StructType([StructField("Tweet content", StringType())\
                     ,StructField("date", StringType())])

In [13]:
#fonction inserer csv qui sert a append les messages(tweet) recu par le produceur dans un fichier csv
def csv_insert(df,f_name) :
    #df spark to df pandas
    df = df_final_csv.toPandas()
    #df pandas to dict
    mon_dict=df.to_dict(orient="record")
    #si le fichier existe on le cree
    if os. path.exists(f_name) :
        with open(f_name, 'a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames= mon_dict[len(mon_dict)-1].keys())
            writer.writerow(mon_dict[len(mon_dict)-1])
    #sinon on append direct sans creer de nouveau fichier
    else:
        with open(f_name, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=mon_dict[len(mon_dict)-1].keys())
            writer.writeheader()
            writer.writerow(mon_dict[len(mon_dict)-1])

In [14]:
#recuperation des messages envoyés par le produceur
for msg in c:
    #message to df pandas
    df = pd.DataFrame(json.loads(msg.value))
    #df pandas to df spark
    dfs = spark.createDataFrame(df,schema=cSchema)
    #tranformation avec la pipe
    df_pipe = pipe.transform(dfs)
    #construction de la df final avec une union des df tranformé a chaque message
    df_final=df_final.union(df_pipe)
    df_final.show()
    #nettoyage de la df final
    df_final_csv= df_final.drop('words','Resultat','Ngram','features','idf','rawPrediction','probability')
    #creation de la colonne sentimenet pour faciliter la visualisation derreire
    df_final_csv = df_final_csv.withColumn("Sentiment", when(df_final_csv.prediction == 1,"Positive")
                                                    .when(df_final_csv.prediction== 0 ,"Negative"))
    df_final_csv.show()
    #Creation et insertion dans la bdd mongo
    df_final_csv.write.format("mongo").mode("append").option("uri","mongodb://mongo:27017/Twitter.sentiment").save()
    #sauvgarde des donnees dans un csv
    csv_insert(df_final_csv,'streamlit.csv')

    

+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|       Tweet content|      date|               words|            Resultat|               Ngram|            features|                 idf|       rawPrediction|         probability|prediction|
+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|RT @ICSCERT: 🎉 @...|09/02/2022|[rt, @icscert:, ?...|[rt, @icscert:, ?...|[rt @icscert:, @i...|(262144,[20800,25...|(262144,[20800,25...|[9.59890765140024...|[0.99993220183659...|       0.0|
|Baxter Healthcare...|08/02/2022|[baxter, healthca...|[baxter, healthca...|[baxter healthcar...|(262144,[723,2080...|(262144,[723,2080...|[17.0373785398440...|[0.99999996011950...|       0.0|
+--------------------+----------+--------

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|       Tweet content|      date|               words|            Resultat|               Ngram|            features|                 idf|       rawPrediction|         probability|prediction|
+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|RT @ICSCERT: 🎉 @...|09/02/2022|[rt, @icscert:, ?...|[rt, @icscert:, ?...|[rt @icscert:, @i...|(262144,[20800,25...|(262144,[20800,25...|[9.59890765140024...|[0.99993220183659...|       0.0|
|Baxter Healthcare...|08/02/2022|[baxter, healthca...|[baxter, healthca...|[baxter healthcar...|(262144,[723,2080...|(262144,[723,2080...|[17.0373785398440...|[0.99999996011950...|       0.0|
|Minutes from the ...|07/02/2022|[minutes

PermissionError: [Errno 13] Permission denied: 'streamlit.csv'